# a. 使用 Transformers 加载量化后的 LLM 大模型（GPTQ & AWQ） 

> 引导文章：[19a. 从加载到对话：使用 Transformers 本地运行量化 LLM 大模型（GPTQ & AWQ）](https://github.com/Hoper-J/AI-Guide-and-Demos-zh_CN/blob/master/Guide/19a.%20从加载到对话：使用%20Transformers%20本地运行量化%20LLM%20大模型（GPTQ%20%26%20AWQ）.md)。

代码文件没有显卡要求，在个人计算机上均可进行对话。

**模型文件约为 4 GB**。

这里还有一个简单的 [🎡 AI Chat 脚本](https://github.com/Hoper-J/AI-Guide-and-Demos-zh_CN/blob/master/CodePlayground/chat.py)供你尝试，详见：[CodePlayground](https://github.com/Hoper-J/AI-Guide-and-Demos-zh_CN/blob/master/CodePlayground/README.md#当前的玩具)，点击 `►` 或对应的文本展开。

Llama-cpp-python 关于 GGUF 文件加载的相关链接：[文章 19b](https://github.com/Hoper-J/AI-Guide-and-Demos-zh_CN/blob/master/Guide/19b.%20从加载到对话：使用%20Llama-cpp-python%20本地运行量化%20LLM%20大模型（GGUF）.md) | [代码文件 16b](https://github.com/Hoper-J/AI-Guide-and-Demos-zh_CN/blob/master/Demos/16b.%20使用%20Llama-cpp-python%20加载量化后的%20LLM%20大模型（GGUF）.ipynb)。

在线链接：[Kaggle - a](https://www.kaggle.com/code/aidemos/16a-transformers-llm-gptq) | [Colab - a](https://colab.research.google.com/drive/1cmIDjHriW8aQ5mIsV6ZeTqdnqYe6PoOv?usp=sharing)

## 手动下载模型（推荐）

来试试多线程指定文件下载，对于 Linux，这里给出配置命令，其余系统可以参照[《a. 使用 HFD 加快 Hugging Face 模型和数据集的下载》](https://github.com/Hoper-J/AI-Guide-and-Demos-zh_CN/blob/master/Guide/a.%20使用%20HFD%20加快%20Hugging%20Face%20模型和数据集的下载.md)先进行环境配置。你也可以跳过这部分，后面会介绍自动下载。

```bash
sudo apt-get update
sudo apt-get install git git-lfs wget aria2
git lfs install
```

下载并配置 HFD 脚本：

```bash
wget https://huggingface.co/hfd/hfd.sh
chmod a+x hfd.sh
export HF_ENDPOINT=https://hf-mirror.com
```

使用多线程下载指定模型。

### GPTQ

命令遵循 `./hfd.sh <model_path> --tool aria2c -x <线程数>`的格式：

```bash
./hfd.sh neuralmagic/Mistral-7B-Instruct-v0.3-GPTQ-4bit --tool aria2c -x 16
```

### AWQ

命令遵循 `./hfd.sh <model_path> --tool aria2c -x <线程数>`的格式：

```python
./hfd.sh solidrust/Mistral-7B-Instruct-v0.3-AWQ --tool aria2c -x 16
```

### GGUF

使用多线程下载指定模型，命令遵循 `./hfd.sh <model_path> --include <file_name> --tool aria2c -x <线程数>`的格式：

```python
./hfd.sh bartowski/Mistral-7B-Instruct-v0.3-GGUF --include Mistral-7B-Instruct-v0.3-Q4_K_M.gguf --tool aria2c -x 16
```

下载完成你应该可以看到类似的输出：

```
Download Results:
gid   |stat|avg speed  |path/URI
======+====+===========+=======================================================
145eba|OK  |   6.8MiB/s|./Mistral-7B-Instruct-v0.3-Q4_K_M.gguf

Status Legend:
(OK):download completed.
Downloaded https://huggingface.co/bartowski/Mistral-7B-Instruct-v0.3-GGUF/resolve/main/Mistral-7B-Instruct-v0.3-Q4_K_M.gguf successfully.
Download completed successfully.
```



## Transformer

### 环境配置

In [ ]:
!pip install numpy==1.24.4
!pip install --upgrade transformers
!pip install optimum

你需要注意的是，如果安装不正确，GPTQ 将无法正确使用 GPU 进行推理，也就是说无法进行加速，即便 print(model.device) 显示为 "cuda"。类似的问题见 [Is This Inference Speed Slow?  #130](https://github.com/AutoGPTQ/AutoGPTQ/issues/130) 和 [CUDA extension not installed #694](https://github.com/AutoGPTQ/AutoGPTQ/issues/694)。

这个问题是普遍存在的，当你直接使用 `pip install auto-gptq` 进行安装时，可能就会出现。

你可以通过以下命令检查已安装的版本：

```bash
pip list | grep auto-gptq
```

如果发现之前安装的版本不带 cuda 标识，卸载它，从源码重新进行安装（推理速度将提升为原来的 15 倍以上）。

```bash
pip uninstall auto-gptq
git clone https://github.com/PanQiWei/AutoGPTQ.git && cd AutoGPTQ
```

```bash
# 以下两种方式任选一种进行安装即可，经测试均有效
pip install -vvv --no-build-isolation -e .
# >> Successfully installed auto-gptq-0.8.0.dev0+cu121

python setup.py install
# >> Finished processing dependencies for auto-gptq==0.8.0.dev0+cu121
```

### GPTQ

#### 导入库

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

下面介绍两种导入模型的方法，实际执行时本地/自动导入二选一。

#### 设置模型路径


In [ ]:
# 如果你已经配置过了，可以直接在 Notebook 中执行下面的命令下载。
!export HF_ENDPOINT=https://hf-mirror.com
!./hfd.sh neuralmagic/Mistral-7B-Instruct-v0.3-GPTQ-4bit --tool aria2c -x 16

如果已经在本地下载了模型，可以通过指定模型路径来加载模型。以下示例假设模型位于当前目录的 `Mistral-7B-Instruct-v0.3-GPTQ-4bit` 文件夹下：

In [ ]:
# 指定本地模型的路径
model_path = "./Mistral-7B-Instruct-v0.3-GPTQ-4bit"

如果没有本地模型，设置远程路径（`id` + `/` + `model_name`），导入的时候会自动从 Hugging Face 下载模型：

In [ ]:
# 指定远程模型的路径
model_path = "neuralmagic/Mistral-7B-Instruct-v0.3-GPTQ-4bit"

#### 加载模型


In [ ]:
# 加载分词器
tokenizer = AutoTokenizer.from_pretrained(model_path)

# 下载并加载模型
model = AutoModelForCausalLM.from_pretrained(
    model_path,
    torch_dtype="auto",  # 自动选择模型的权重数据类型
    device_map="auto"    # 自动选择可用的设备（CPU/GPU）
)

#### 推理测试

In [ ]:
# 输入文本
input_text = "Hello, World!"

# 将输入文本编码为模型可接受的格式
input_ids = tokenizer.encode(input_text, return_tensors="pt").to(model.device)

# 生成输出
with torch.no_grad():
    output_ids = model.generate(
        input_ids=input_ids,
        max_length=50,
        pad_token_id=tokenizer.eos_token_id
    )

# 解码生成的输出
output_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)

# 打印生成的文本
print(output_text)

### AWQ

In [ ]:
!pip install autoawq autoawq-kernels

#### 导入库

In [ ]:
import torch
from awq import AutoAWQForCausalLM
from transformers import AutoTokenizer, TextStreamer

下面介绍两种导入模型的方法，实际执行时本地/自动导入二选一。

#### 设置模型路径


In [ ]:
# 如果你已经配置过了，可以直接在 Notebook 中执行下面的命令下载。
!export HF_ENDPOINT=https://hf-mirror.com
!./hfd.sh solidrust/Mistral-7B-Instruct-v0.3-AWQ --tool aria2c -x 16

如果已经在本地下载了模型，可以通过指定模型路径来加载模型。以下示例假设模型位于当前目录的 `Mistral-7B-Instruct-v0.3-AWQ` 文件夹下：

In [ ]:
# 指定本地模型的路径
model_path = "./Mistral-7B-Instruct-v0.3-AWQ"

如果没有本地模型，设置远程路径（`id` + `/` + `model_name`），导入的时候会自动从 Hugging Face 下载模型：


In [ ]:
# 指定远程模型的路径
model_path = "solidrust/Mistral-7B-Instruct-v0.3-AWQ"

#### 加载模型

一些权重不会被加载，对于当前任务来说这是预期的行为。

In [ ]:
# 加载分词器
tokenizer = AutoTokenizer.from_pretrained(
    model_path,
    trust_remote_code=True
)

# 下载并加载模型
model = AutoAWQForCausalLM.from_quantized(
    model_path,
    fuse_layers=True  # 融合部分模型层以提高推理速度
)

#### 推理测试

In [ ]:
# 设置设备
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# 输入文本
input_text = "Hello, World!"

# 将输入文本编码为模型可接受的格式
input_ids = tokenizer.encode(input_text, return_tensors="pt").to(device)

# 生成输出
with torch.no_grad():
    output_ids = model.generate(
        input_ids=input_ids,
        max_length=50,
        pad_token_id=tokenizer.eos_token_id
    )

# 解码生成的输出
output_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)

# 打印生成的文本
print(output_text)

### 统一方式加载

值得一提的是，你可能已经注意到了 GPTQ 并不是使用 Auto-GPTQ 库，而是直接使用 Transformers（虽然背后仍然依赖于 Auto-GPTQ）。

“要是所有的模型都能统一就好了，这样就不用查阅其他库的文档了。”

事实上，AWQ 也可以通过 Transformers 来加载（如果你对参数设置没有更细致的要求）：

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

# 指定模型，切换为对应的 GPTQ 或 AWQ 模型路径，可以是远程路径，会自动下载
model_path = "neuralmagic/Mistral-7B-Instruct-v0.3-GPTQ-4bit"

# 加载分词器
tokenizer = AutoTokenizer.from_pretrained(model_path)

# 加载模型
model = AutoModelForCausalLM.from_pretrained(
    model_path,
    torch_dtype="auto",  # 自动选择模型的权重数据类型
    device_map="auto"    # 自动选择可用的设备（CPU/GPU）
)

# 设置设备
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# 输入文本
input_text = "Hello, World!"

# 将输入文本编码为模型可接受的格式
input_ids = tokenizer.encode(input_text, return_tensors="pt").to(device)

# 生成输出
with torch.no_grad():
    output_ids = model.generate(
        input_ids=input_ids,
        max_length=50,
        pad_token_id=tokenizer.eos_token_id
    )

# 解码生成的输出
output_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)

# 打印生成的文本
print(output_text)

### 了解提示词模版（prompt_template）

其实非常简单，就是曾经提到的占位符（下图对于 `{{question}}` 的应用）。

![占位符](../Guide/assets/%E5%8D%A0%E4%BD%8D%E7%AC%A6-6055722.png)

举个直观的例子：

In [ ]:
# 定义 Prompt Template
prompt_template = "问：{question}\n答："

# 定义问题
question = "人工智能的未来发展方向是什么？"

# 使用 Prompt Template 生成完整的提示
prompt = prompt_template.format(question=question)
print(prompt)
# print("\n")
# print(f"问：{question}\n答：")

#### tokenizer.chat_template

查看模型的 `chat_template`。

In [ ]:
# 打印 chat_template 信息（如果存在的话）
if hasattr(tokenizer, 'chat_template'):
    print(tokenizer.chat_template)
else:
    print("Tokenizer 没有 'chat_template' 属性。")

### 流式输出

在项目初期认识 API 的时候，文章[《01. 初识 LLM API：环境配置与多轮对话演示》](https://github.com/Hoper-J/AI-Guide-and-Demos-zh_CN/blob/master/Guide/01.%20初识%20LLM%20API：环境配置与多轮对话演示.md#流式输出)有提到过流式输出，这是我们一直以来见到的大模型输出方式：逐字（token）打印而非等全部生成完打印。

执行下面的代码试试（无论之前导入的是哪种模型，都可以继续）：

In [ ]:
from transformers import TextStreamer

device = 'cuda' if torch.cuda.is_available() else 'cpu'

# 创建 TextStreamer 实例
streamer = TextStreamer(
    tokenizer, 
    skip_prompt=True,         # 在输出时跳过输入的提示部分，仅显示生成的文本
    skip_special_tokens=True  # 忽略生成过程中的特殊标记（比如 <pad> / <eos> ...）
)

# 将提示编码为模型输入
input_ids = tokenizer.encode(prompt, return_tensors="pt").to(device)

# 设置生成参数
generation_kwargs = {
    "input_ids": input_ids,  # 模型的输入 ID，注意，这不是 Embedding
    "max_length": 200,       # 生成的最大 token 数
    "streamer": streamer,    # 使用 TextStreamer 实现生成过程中逐步输出文本
    "pad_token_id": tokenizer.eos_token_id  # 默认行为，消除 open-end 警告
}

# 开始生成文本
with torch.no_grad():
    # ** 是 Python 中的解包操作符，它将字典中的键值对解包为函数的关键字参数。
    # 在这里，**generation_kwargs 将字典中的参数逐一传递给 model.generate() 方法，
    # 等效于直接写出所有参数：
    # model.generate(input_ids=input_ids, max_length=200, do_sample=True, ...)
    # 你需要注意到，这和之前采用了不同的传参方式，但本质是一样的。
    # 在后续的教程中，会较少地使用这种方式进行传参。
    # 因为这很好的分离了参数，所以也增加了乍一看之下的抽象度，为了初见的直观，将减少使用。
    model.generate(**generation_kwargs)

### 单轮对话


（如果重新启动内核的话，遵循 `导入库`-> `导入模型` -> `当前代码块` 的顺序执行。）

让我们直接设计 `messages`，并应用 `chat_template` 进行对话：


In [ ]:
from transformers import TextStreamer

device = 'cuda' if torch.cuda.is_available() else 'cpu'

# 定义输入
prompt = input("User: ")

# 定义消息列表
messages = [
    {"role": "user", "content": prompt}
]

# 使用 tokenizer.apply_chat_template() 生成模型输入
input_ids = tokenizer.apply_chat_template(messages, return_tensors="pt").to(device)

# 创建 TextStreamer 实例
streamer = TextStreamer(
    tokenizer, 
    skip_prompt=True,         # 在输出时跳过输入的提示部分，仅显示生成的文本
    skip_special_tokens=True  # 忽略生成过程中的特殊标记（比如 <pad> / <eos> ...）
)

# 设置生成参数
generation_kwargs = {
    "input_ids": input_ids,  # 模型的输入 ID，注意，这不是 Embedding
    "max_length": 500,      # 生成的最大 token 数
    "streamer": streamer,    # 使用 TextStreamer 实现生成过程中逐步输出文本
    "pad_token_id": tokenizer.eos_token_id  # 默认行为，消除 open-end 警告
}

# 开始生成文本
with torch.no_grad():
    model.generate(**generation_kwargs)

### 多轮对话

如果重新启动内核的话，遵循 `导入库`-> `导入模型` -> `当前代码块` 的顺序执行。

In [ ]:
from transformers import TextStreamer

device = 'cuda' if torch.cuda.is_available() else 'cpu'

# 初始化对话历史
messages = []

# 开始多轮对话
while True:
    # 获取输入
    prompt = input("User: ")
    
    # 退出对话条件（当然，你也可以直接终止代码块）
    if prompt.lower() in ["exit", "quit", "bye"]:
        print("Goodbye!")
        break
    
    # 将输入添加到对话历史
    messages.append({"role": "user", "content": prompt})
    
    # 使用 tokenizer.apply_chat_template() 生成模型输入
    input_ids = tokenizer.apply_chat_template(messages, return_tensors="pt").to(device)
    
    # 创建 TextStreamer 实例
    streamer = TextStreamer(
        tokenizer, 
        skip_prompt=True,         # 在输出时跳过输入的提示部分，仅显示生成的文本
        skip_special_tokens=True  # 忽略生成过程中的特殊标记（比如 <pad> / <eos> ...）
    )
    
    # 设置生成参数
    generation_kwargs = {
        "input_ids": input_ids,                  # 模型的输入 ID
        "max_length": input_ids.shape[1] + 500,  # 生成的最大 token 数，input_ids.shape[1] 即输入对应的 tokens 数量
        "streamer": streamer,                    # 使用 TextStreamer 实现生成过程中逐步输出文本
        "pad_token_id": tokenizer.eos_token_id   # 默认行为，消除警告
    }
    
    # 开始生成回复
    with torch.no_grad():
        output_ids = model.generate(**generation_kwargs)
    
    # 获取生成的回复文本
    assistant_reply = tokenizer.decode(output_ids[0][input_ids.shape[1]:], skip_special_tokens=True)
    
    # 将模型的回复添加到对话历史
    messages.append({"role": "assistant", "content": assistant_reply})

注意，这里有一个小坑，你不能简单使用 `output_ids[0]` 来保存回复，因为`output_ids` 中实际上包含了 `input_ids`，打印它们：


In [ ]:
print(tokenizer.decode(input_ids[0], skip_special_tokens=True))
print(tokenizer.decode(output_ids[0], skip_special_tokens=True))